In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# veri kaynağı: http://veribilimi.co/data/iris.csv
df = pd.read_csv("D:\\OneDrive\\egitim\\verilen\\bogazici-compec-python-ml\\Datasets\\iris.csv")

# 1. Veri Keşfi

In [3]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [5]:
df.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


# 2. Veri Temizliği ve Ön Hazırlığı

In [6]:
# Veri önceden temizlenmiş olduğundan bu aşamada birşey yapmıyoruz.

## 2.1. Veriyi X,y olarak bölme (nitelikler matrisi ve hedef değişken - dikey bölme)

In [20]:
X = df.iloc[:,:-1].values
X.shape

(150, 4)

In [21]:
y = df.iloc[:,-1].values
y.shape

(150,)

# Kategorik Nitelik Dönüştürme

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
labelencoder = LabelEncoder()

In [24]:
X[:,-1] = labelencoder.fit_transform(X[:,0])

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=142)

In [27]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


# Feature Scaling

In [28]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Model Creation and Training

In [29]:
from sklearn.svm import SVC

In [30]:
classifier = SVC(random_state=142, gamma='auto')

# Computing cross-validated metrics

In [31]:
from sklearn.model_selection import cross_val_score

In [32]:
scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='accuracy')

In [33]:
scores

array([0.91666667, 0.86956522, 0.95652174, 0.80952381, 0.9047619 ])

In [34]:
scores.mean()

0.8914078674948241

In [35]:
 print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.8914 (+/- 0.0989)


# Optimal hiperparametreleri bulma

In [26]:
classifier.get_params

<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=142, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)>

In [27]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

In [39]:
from sklearn.model_selection import GridSearchCV

In [40]:
param_grid = [
  {
    'C': [1.0, 10.0, 100.0,1000.0], 
   'kernel': ['linear']
    },
    {
     'C': [1.0, 10.0, 100.0,1000.0], 
   'kernel': ['rbf'],
   'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]
    }
 ]

In [41]:
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=param_grid,
                          scoring='accuracy',
                          cv=10,
                          n_jobs=4)
# maksimum için n_jobs=-1 kullanılır.

In [42]:
grid_search = grid_search.fit(X_train, y_train)

D:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [43]:
best_score = grid_search.best_score_
best_score

0.9910714285714286

In [44]:
best_params = grid_search.best_params_
best_params

{'C': 10.0, 'kernel': 'linear'}

In [45]:
best_estimator = grid_search.best_estimator_
best_estimator

SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=142, shrinking=True,
  tol=0.001, verbose=False)

# Model kaydetme

In [48]:
from sklearn.externals import joblib

In [49]:
joblib.dump(best_estimator, "best_estimator.pkl")

['best_estimator.pkl']

# Model Okuma

In [50]:
best_estimator_loaded = joblib.load("best_estimator.pkl")

# Predictions

In [51]:
y_pred = best_estimator_loaded.predict(X_test)

## Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [52]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

,y_pred,y_test
0,Iris-setosa,Iris-setosa
1,Iris-versicolor,Iris-versicolor
2,Iris-versicolor,Iris-versicolor
3,Iris-virginica,Iris-virginica
4,Iris-versicolor,Iris-versicolor


# Hata Matrisi (Confusion Matrix)

In [53]:
from sklearn.metrics import confusion_matrix

In [54]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[ 7  0  0]
 [ 0 15  2]
 [ 0  2 12]]


In [55]:
from sklearn.metrics import accuracy_score

In [56]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      89.47


# Elle Accuracy hesaplama

In [57]:
errors = predictions[predictions['y_pred'] != predictions['y_test']]
errors.head()

,y_pred,y_test
8,Iris-versicolor,Iris-virginica
14,Iris-virginica,Iris-versicolor
18,Iris-versicolor,Iris-virginica
19,Iris-virginica,Iris-versicolor


In [58]:
predictions[predictions['y_pred'] != predictions['y_test']].count()

y_pred    4
y_test    4
dtype: int64

In [59]:
len(y_test)

38

In [60]:
accuracy = (len(y_test) - len(errors)) / len(y_test)
accuracy

0.8947368421052632